In [1]:
import torch
import torch.nn as nn

D:\anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Mymodule(nn.Module):
    def __init__(self):
        super(Mymodule, self).__init__()
        self.data = torch.zeros((3))

In [12]:
model = Mymodule()
print('parameters:', list(model.parameters()))
print('state_dict:', model.state_dict())

parameters: []
state_dict: OrderedDict()


## register_parameter在model中注册一个torch.nn.Parameter()，可以在model.parameters()和model.state_dict()显示，

In [4]:
model.register_parameter('p1', nn.Parameter(torch.zeros((2))))
print('parameters:', list(model.parameters()))
print('state_dict:', model.state_dict())

print('get_parameter:', model.get_parameter('p1'))
print('named_parameters:', list(model.named_parameters()))

parameters: [Parameter containing:
tensor([0., 0.], requires_grad=True)]
state_dict: OrderedDict([('p1', tensor([0., 0.]))])
get_parameter: Parameter containing:
tensor([0., 0.], requires_grad=True)
named_parameters: [('p1', Parameter containing:
tensor([0., 0.], requires_grad=True))]


## persistent为True时，buffer会在model.state_dict()中；反之则反。

In [6]:
model.register_buffer('b1', torch.zeros((2)), persistent=False)
print('parameters:', list(model.parameters()))
print('state_dict:', model.state_dict())

parameters: []
state_dict: OrderedDict()


In [8]:
model.register_buffer('b2', torch.zeros((2)), persistent=True)
print('parameters:', list(model.parameters()))
print('state_dict:', model.state_dict())

parameters: []
state_dict: OrderedDict([('b2', tensor([0., 0.]))])


In [9]:
class Submodel1(nn.Module):
    def __init__(self):
        super(Submodel1, self).__init__()
        self.sp1 = nn.Parameter(torch.zeros((3)))
            
submodel1 = Submodel1()
submodel2 = nn.Sequential(nn.Linear(2, 2))

## 向模型中添加Submodule类型的子模块，并且输出

In [11]:
model.add_module('submodel1', submodel1)
print('parameters:', list(model.parameters()))
print('state_dict:', model.state_dict())

print('get_module:', model.get_submodule('submodel1'))
print('named_children:', list(model.named_children()))
print('named_modules:', list(model.named_modules()))

parameters: [Parameter containing:
tensor([0., 0., 0.], requires_grad=True)]
state_dict: OrderedDict([('submodel1.sp1', tensor([0., 0., 0.]))])
get_module: Submodel1()
named_children: [('submodel1', Submodel1())]
named_modules: [('', Mymodule(
  (submodel1): Submodel1()
)), ('submodel1', Submodel1())]


## 向模型中添加Sequential类型的子模块，并且输出

In [13]:
model.add_module('submodel2', submodel2)
print('parameters:', list(model.parameters()))
print('state_dict:', model.state_dict())

print('get_module:', model.get_submodule('submodel2'))
print('named_children:', list(model.named_children()))
print('named_modules:', list(model.named_modules()))

parameters: [Parameter containing:
tensor([[-0.1705, -0.6878],
        [-0.6807,  0.3237]], requires_grad=True), Parameter containing:
tensor([-0.5274,  0.3852], requires_grad=True)]
state_dict: OrderedDict([('submodel2.0.weight', tensor([[-0.1705, -0.6878],
        [-0.6807,  0.3237]])), ('submodel2.0.bias', tensor([-0.5274,  0.3852]))])
get_module: Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
)
named_children: [('submodel2', Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
))]
named_modules: [('', Mymodule(
  (submodel2): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
  )
)), ('submodel2', Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
)), ('submodel2.0', Linear(in_features=2, out_features=2, bias=True))]


## 向模型中注册钩子函数

In [14]:
def forward_hook(target_layer,input, output):
    print(list(target_layer.parameters()))
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cnn = nn.Conv2d(in_channels=2, out_channels=4, kernel_size=(3, 3))
    def forward(self, inputs):
        out = self.cnn(inputs)
        return out

In [15]:
inputs = torch.randn((2, 2, 4, 4))
model = Model()
model.cnn.register_forward_hook(forward_hook)
out = model(inputs)
# print(out)

[Parameter containing:
tensor([[[[ 0.2212, -0.2307, -0.1123],
          [ 0.0096,  0.2092,  0.1990],
          [ 0.1003,  0.1667,  0.2103]],

         [[ 0.0260, -0.2198, -0.0540],
          [ 0.2053,  0.1322,  0.0139],
          [ 0.2082,  0.0973,  0.0750]]],


        [[[ 0.1326,  0.0301,  0.2125],
          [-0.1453,  0.1786,  0.1431],
          [-0.0130, -0.2326,  0.1381]],

         [[-0.0576,  0.0096, -0.0003],
          [-0.0596,  0.1225,  0.0084],
          [ 0.2201, -0.0655,  0.1831]]],


        [[[-0.0548, -0.2264,  0.1947],
          [-0.0857,  0.0988,  0.1814],
          [-0.1266,  0.1805,  0.0316]],

         [[-0.0090,  0.1855, -0.1060],
          [ 0.0259,  0.1473, -0.0550],
          [-0.0258,  0.1454, -0.1527]]],


        [[[ 0.1151,  0.0235, -0.1572],
          [-0.1242, -0.1021, -0.1790],
          [ 0.0626, -0.1168, -0.0263]],

         [[ 0.1644,  0.0357,  0.0300],
          [ 0.0640,  0.0385, -0.0118],
          [-0.1499,  0.0441, -0.0263]]]], requires_grad=True